## Coroutine termination and exception handling

An unhandled exception within a coroutine propagates to the caller of the `next` or `send` which triggered it.

In [1]:
def averager():
    total = 0
    count = 0
    average = None
    while True:
        total += yield average
        count += 1
        average = total/count

In [3]:
coro_avg = averager()
next(coro_avg)

In [5]:
coro_avg.send(40)

40.0

In [6]:
coro_avg.send(30)

35.0

In [8]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [9]:
coro_avg.send(10)

StopIteration: 

Because the exception was not handled in the coroutine, it terminated. Any attempt to reactivate it will raise `StopIteration`

There are two methods for explicitly sending exceptions into the coroutine - `throw` and `close`

- `generator.throw(exc_type[, exc_value[, traceback]])`
Causes the `yield` expression where the generator was paused to raise the exception.

- `generator.close()`
Causes the `yield` expression where the generator was paused to raise a `Generator Exit` exception or raises `StopIteration`.

In [10]:
class DemoException(Exception):
    """An exception for the demonstration."""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> corutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run')

In [11]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [12]:
exc_coro.send(11)

-> corutine received: 11


In [13]:
exc_coro.send(22)

-> corutine received: 22


In [14]:
exc_coro.close()

In [15]:
from inspect import getgeneratorstate

In [16]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [17]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [19]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [20]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [21]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [22]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [23]:
# Performs actions on coroutine termination

def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending.')